In [11]:
import pandas as pd

df=pd.read_csv("hyponym-hypernym-pairs-with-snippets.csv", usecols=range(0, 3))
df.head(25) 
#df= pd.read_excel('hyponym-hypernym-pairs-with-snippets-csv.csv', sheet_name='TDSheet', usecols=['Поставщик', 'Номенклатура контрагента в ЭСФ','Номенклатура в 1С'])
#df.head(25)

,HYPONYM,HYPERNYM,SNIPPET
0,МОТОРКА,лодка,Моторная лодка.
1,МОТОРКА,моторная лодка,Моторная лодка.
2,МОТОРКА,мотор,Моторная лодка.
3,МОТОРКА,лодка-самоходка.,Лодка-самоходка
4,МОТОРКА,транспортное средство,Моторка как транспортное средство
5,МОТОФРИСТАЙЛ,прыжки c акробатическими элементами,"Мотофристайл (англ. Freestyle motocross, Frees..."
6,МОТОФРИСТАЙЛ,спорт,"Мотофристайл (англ. Freestyle motocross, Frees..."
7,МОТОФРИСТАЙЛ,трюки,"Мотофристайл (англ. Freestyle motocross, Frees..."
8,МОТОФРИСТАЙЛ,соревнование,"Мотофристайл (фристайл мотокросс, freestyle mo..."
9,МОТОФРИСТАЙЛ,Фристайл мотокросс,"Мотофристайл (фристайл мотокросс, freestyle mo..."


In [12]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

In [13]:
START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [31]:
def get_hyperonyms(row):
    
    main_word = row['HYPONYM']
    text = row['SNIPPET']
    
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule (END, MID, HYPONYM),
        rule(END, MID, MID, END),
        rule(HYPONYM, END),
        rule(END, MID, END),
        rule (HYPONYM, KAK, END),
        rule (END, KAK, HYPONYM)
        
    )
    parser = Parser(RULE) 
    text = utils.deaccent(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower()
    hypernyms = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hypernyms.append(k)
    print(main_word, hypernyms)
    return hypernyms

In [32]:
df['get_hyperonyms'] = df.apply(lambda row: get_hyperonyms(row), axis=1)

МОТОРКА []
МОТОРКА []
МОТОРКА []
МОТОРКА []
МОТОРКА []
МОТОФРИСТАЙЛ [['мотофристаил', '—', 'это', 'термин'], ['мотофристаил', '—', 'это', 'молодои', 'вид', 'спорта']]
МОТОФРИСТАЙЛ [['мотофристаил', '—', 'это', 'термин'], ['мотофристаил', '—', 'это', 'молодои', 'вид', 'спорта']]
МОТОФРИСТАЙЛ [['мотофристаил', '—', 'это', 'термин'], ['мотофристаил', '—', 'это', 'молодои', 'вид', 'спорта']]
МОТОФРИСТАЙЛ [['мотофристаил', '—', 'в', 'этом', 'соревновании']]
МОТОФРИСТАЙЛ [['мотофристаил', '—', 'в', 'этом', 'соревновании']]
МОХОВИК [['моховик', '—', 'гриб']]
МОХОВИК [['моховик', '—', 'род', 'съедобных', 'трубчатых', 'грибов']]
МОХОВИК [['моховик', '—', 'род', 'съедобных', 'трубчатых', 'грибов']]
МОХОВИК [['моховик', '—', 'род', 'съедобных', 'трубчатых', 'грибов']]
МОХОВИК []
МОШКА []
МОШКА []
МОШКА []
МОШКА [['гнус', '—', 'обобщенное', 'название', 'скоплении']]
МОШКА [['гнус', '—', 'обобщенное', 'название', 'скоплении']]
МУЗЕЕВЕД []
МУЗЕЕВЕД []
МУЗЕЕВЕД []
МУЗЕЕВЕД []
МУЗЕЕВЕД []
МУЗЕЕВЕДЕНИЕ